# Predicir la extensión de incendios forestales con bosques aleatorios

## Introducción

Incendios ocurren a menudo en parques naturales y tienen un impacto grande en la sustentabilidad de la flora y fauna y la seguridad de los visitantes. Por este motivo, se quiere entender los factores que son relevantes para el fenómeno de los incendios forestales. Este tutorial pretende de diseñar un modelo de *machine learning* para la predicción de la extensión de incendios forestales.

La tutorial está basada en el artícula "*A Data Mining Approach to Predict Forest Fires using Meteorological Data*" de P. Cortez y A. Morais (ver http://archive.ics.uci.edu/ml/datasets/Forest+Fires).

## Cargar la base de datos

Antes de comenzar el análisis, tenemos que cargar la base de datos con los registros de incendios históricos del parque nacional *Montesinho* en el norte del Portugal. La base de datos está disponible en internet (ver http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv) en formato CSV. Julia puede cargar bases de datos directamente del internet con el paquete *HTTP*. El paquete *CSV* es necesario para abrir archivos formato CSV y un paquete adecuado para manejar bases de datos en Julia es *DataFrames*.

Primero, tenemos que cargar los paquetes en Julia con la palabra reservada `using`.

In [ ]:
using CSV, DataFrames, HTTP

Si recibes un error que Julia no puede encontrar el paquete, capaz que nunca has usada estos paquetes antes. Para usar nuevos paquetes, se tiene que instalarlos con el paquete *Pkg* y la función `Pkg.add("paquete")`.

In [ ]:
# using Pkg
# Pkg.add(["CSV", "DataFrames", "HTTP"])

Ahora cargamos la base de datos del internet y lo abrimos como un objeto de *DataFrame*.

In [ ]:
datafile = HTTP.get("https://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv")
df = CSV.read(IOBuffer(datafile.body))

Si quieres abrir una base de datos desde tu computador, tienes que guardarla en la mapa del código y usar la función `CSV.read()'.

In [ ]:
# df = CSV.read("forestfires.csv")

La función previa mostró el primer parte de la base de datos. Para explorar la base de datos, responde las preguntas siguientes.

**Ejercicio:** Muestra los primeros 10 registros de la base de datos con la función `first()`. La primera variable de *input* is el *Dataframe* y la segunda el número de filas para mostrar.

Recuerde que se puede ver la documentación de una función con un ?, es decir, ```?first```.

**Ejercicio:** Muestra los últimos 7 registros de la base de datos con la función `last()`.

**Ejercicio:** ¿Cuántos registros hay en la base? ¿Cuántos variables tiene cada registro? *Sugerencia:* Usa la función `size()`.

**Ejercicio:** Guarda los nombres de los variables en un arreglo con la función `names()`.

**Ejercicio:** Usa la función `describe()` y explica el resultado.

## Explorar la base de datos

Ahora tenemos la base de datos cargado y corriendo. Queremos entender mas sobre los registros en la base y el fenómeno de incendios forestales. Por este motivo vamos a dibujar algunos estadísticas de las variables.

La variable clave en la base de datos es el área quemada, lo cual va ser la variable que queremos predecir, el *label*. Los *features* son las variables que podrían tener valor predictivo, por ejemplo la temperatura y humedad.

Primero, tomamos los datos del área quemada de cada registro y lo guardamos en un arreglo.

**Ejercicio:** Explica que pasa en el código siguiente.

In [ ]:
areaIncendio = convert.(Float64, dropmissing(df).area)

**Ejercicio:** Calcule el mínimo, máximo, promedio y mediano del áerea quemada.

*Sugerencia:* El paquete ```Statistics``` podría ser útil, porque tiene las funciones ```minimum```, ```maximum```, ```mean``` y ```mediano```.

La unidad de área quemada es hectários.

Para entender la distribución del área de los incendios históricos, una diagrama de baras sería valioso. Para plotear, Julia tiene una variedad de paquetes, uno de los cuales es ```Plots```. Este paquete es una interfaz a distintos *backends*, como por ejemplo ````GR```.

In [ ]:
using Plots
gr()

In [ ]:
histogram(areaIncendio, bins=5, label="", title="Cantidad de incendios forestales", xlabel="área quemada")

**Ejercico:** Cambia el número de franjas para mejorar la histograma.

Se nota que hay muchos incendios pequeños en la base de datos. Una cantidad grande de registros tiene 0.0 como área quemada. Esto dice que el área era más pequeña que la escala mínima para registarlo, entonces, ¡sí había un incendio! Para visualisar la magnitud de los incendios mejor, se puede usar una escala logarítmica.

In [ ]:
logArea = log10.(areaIncendio.+1);

In [ ]:
histogram(logArea, bins=5, label="", title="Cantidad de incendios forestales", xlabel="log10(área quemada)")

**Ejercicio:** Explica por qué se tiene que sumar 1 al área quemada antes de tomar el logarítmo.

**Ejercicio:** Explica por qué la histograma es distinta después tomar el logarítmo.

## Buscar variables con valor predictivo

Para evitar futuros incendios forestales grandes, es importante entender cuales variables son importantes a la hora de cuantificar el riesgo para tener incendios. Por ejemplo, una temperatura alta podría apoyar en la propagación de incendios forestales. Para ver si eso fuera así en los datos históricos, hacemos una exploración de las relaciones entre los variables y el área quemada de incendios históricos.

La base de datos tiene cuatro variables de la clima cuando el incendio surgió. Dibujamos estos variables con respecto al área quemada.

In [ ]:
temperatura = convert.(Float64, dropmissing(df).temp);
humedad = convert.(Float64, dropmissing(df).RH);
viento = convert.(Float64, dropmissing(df).wind);
lluvia = convert.(Float64, dropmissing(df).rain);

In [ ]:
pl1 = scatter(temperatura, logArea, label="", xlabel="temperatura", ylabel="log(área quemada)");
pl2 = scatter(humedad, logArea, label="", xlabel="humedad", ylabel="log(área quemada)");
pl3 = scatter(viento, logArea, label="", xlabel="viento", ylabel="log(área quemada)");
pl4 = scatter(lluvia, logArea, label="", xlabel="lluvia", ylabel="log(área quemada)");
plot(pl1,pl2,pl3,pl4)

**Ejercicio:** Explica cuál de las variables tiene un valor predictivo mayor.

Las agencias de meteorología han definida distintas medidas para el riesgo de incendios forestales, como por ejemplo el *Forest Weather Index* (FWI). Estos índices principalmente dependen de los factores *Fine Fuel Moisture Code* (FFMC), *Duff Moisture Code* (DMC), *Drought Code* (DC), y *Initial Spread Index* (ISI).

**Ejercicio:** Dibuja la relación entre estos indicadores y el área quemada de los incendios en la base de datos.

*Sugerencia:* Se puede copiar el código para plotear las variables de clima y cambiar los nombres.

La base de datos también cuenta con datos como la fecha y ubicación del incendio.

**Ejercicio:** Investigue si la fecha y ubicación tiene impacto en el área quemada y explica por qué sería así.

Para ver el efecto del día o mes, un *boxplot* podría ser interesante.

In [ ]:
using StatsPlots

In [ ]:
pl1 = boxplot(mes, logArea, label="", ylabel="log(área quemada)")
pl2 = boxplot(día, logArea, label="", ylabel="log(área quemada)")
plot(pl1,pl2)

## Crear modelos predictivos con aprendizaje automatizado

Dado que se puede medir los variables cada día, sería valioso tener un modelo que puede decir algo sobre el riesgo de tener un incendio en cualquier día. Los modelos matemáticos pueden buscar patrones en los datos históricos y usar estos patrones para predicir el futuro. Pero antes de usar modelos predictivos, necesitamos entender la precisión del modelo también. Esto podemos lograr con un testeo del modelo con un parte de los registros. Por este motivo, tenemos que particionar la base de datos en un conjunto para entrenamiento y un conjunto para el testeo.

Normalmente se usa 80% de los datos para entrener y los otros 20% para el testeo.

In [ ]:
nEntrenar = trunc(Int,0.8*sizeDF[1])
println("El número de registros para el entrenamiento es ",nEntrenar)
println("El número de registros para el testeo es ",sizeDF[1]-nEntrenar)

**Ejercicio:** Explica que pasó en el código anterior.

Para ambos el conjunto de entrenamiento y el conjunto de testeo tenemos que buscar los datos para la predicción (*features*) y los datos del resultado (etiquetas), lo cual es el área quemada.

In [ ]:
datos_entrenar = convert(Array, dropmissing(df)[1:nEntrenar,1:12]);
etiquetas_entrenar = convert(Array, dropmissing(df)[1:nEntrenar,13]);

datos_testeo = convert(Array, dropmissing(df)[nEntrenar+1:sizeDF[1],1:12]);
etiquetas_testeo = convert(Array, dropmissing(df)[nEntrenar+1:sizeDF[1],13]);

Como modelo de predicción usamos bosques aleatorios (*random forests*).

In [ ]:
using DecisionTree

In [ ]:
modelo = build_forest(etiquetas_entrenar, datos_entrenar)

**Ejercicio:** Explica el *output* que da la función.

Se puede analizar uno de los árboles de decisión con la función `print_tree()`.

In [ ]:
print_tree(modelo.trees[1])

**Ejercicio:** ¿Cuál es la primera pregunta que haga este árbol de decisión? ¿Y el segundo árbol?

Ahora que el modelo está entrenado, se puede hacer las predicciones para el conjunto de testeo.

In [ ]:
predicciones = [apply_forest(modelo, datos_testeo[i, :]) for i = 1:size(datos_testeo)[1]];

In [ ]:
scatter([etiquetas_testeo, predicciones], xlabel="registro", ylabel="área quemada", label=["realidad","predicción"])

In [ ]:
error = abs.(etiquetas_testeo-predicciones);

In [ ]:
histogram(error, bins=15, label="", title="Errores de la predicción", xlabel="área quemada")

**Ejercicio:** Calcula el promedio del error.

**Ejercicio:** Cambia las etiquetas en el logarítmo del área quemada y analiza el rendimiento.

*Sugerencia:* Se puede copiar el código anterior y cambiar los nombres y variables.

**Ejercicio:** Calcula el error con signo, es decir, elimina el valor absoluto en la definición del error. Visualiza la histograma del error, para ambos el área y el logarítmo del área, y explica el resultado.

**Ejercicio:** Explica si es mejor usar el área quemada o el logarítmo del área a la hora de predicir la extensión de incendios forestales.